In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn import preprocessing
import warnings

warnings.filterwarnings("ignore")

import time

In [15]:
data = pd.read_csv('./data/round1_ijcai_18_train_20180301.txt', sep=' ', encoding='utf-8')
data.head()

,instance_id,item_id,item_category_list,item_property_list,item_brand_id,item_city_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,...,context_page_id,predict_category_property,shop_id,shop_review_num_level,shop_review_positive_rate,shop_star_level,shop_score_service,shop_score_delivery,shop_score_description,is_trade
0,108641074714126964,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,4006,5799347067982556520:-1;509660095530134768:-1;5...,6765930309048922341,4,1.0,5002,1.0,1.0,1.0,0
1,5754713551599725161,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,4001,5799347067982556520:9172976955054793469;790838...,6765930309048922341,4,1.0,5002,1.0,1.0,1.0,0
2,842679481291040981,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,4001,5799347067982556520:5131280576272319091;725801...,6765930309048922341,4,1.0,5002,1.0,1.0,1.0,0
3,937088850059189027,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,4016,509660095530134768:-1;5799347067982556520:-1;7...,6765930309048922341,4,1.0,5002,1.0,1.0,1.0,0
4,7975697065017708072,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,4001,5799347067982556520:9172976955054793469;790838...,6765930309048922341,4,1.0,5002,1.0,1.0,1.0,0


In [16]:
def timestamp_datetime(value):
    format = '%Y-%m-%d %H:%M:%S'
    value = time.localtime(value)
    dt = time.strftime(format, value)
    return dt

In [20]:
def base_process(data):
    lbl = preprocessing.LabelEncoder()
    print(
        '----------------------------------------------item-----------------------------------------------')
    ## 计算每个样本的item_category_list和item_property_list中条目长度
#     data['len_item_category'] = data['item_category_list'].map(lambda x: len(str(x).split(';')))  # 长度都为3，但是第一个都是一样的
#     data['len_item_property'] = data['item_property_list'].map(lambda x: len(str(x).split(';')))  # 长度不同
#     for i in range(1, 3):
#         data['item_category_list' + str(i)] = lbl.fit_transform(data['item_category_list'].map(
#             lambda x: str(str(x).split(';')[i]) if len(str(x).split(';')) > i else ''))  # item_category_list的第0列全部都一样
#     for i in range(10):
#         data['item_property_list' + str(i)] = lbl.fit_transform(data['item_property_list'].map(lambda x: str(str(x).split(';')[i]) if len(str(x).split(';')) > i else ''))
    for col in ['item_id', 'item_brand_id', 'item_city_id']:
        ## 使用labelEncoder需要考虑一个问题:
        ## 就是训练集中有的而在测试集中没有以及在测试集中有而在训练集中没有的item_id/item_brand_id/item_city_id
        data[col] = lbl.fit_transform(data[col])
    
#     print(
#         '--------------------------------------------------------------user--------------------------------------------------------------')
#     for col in ['user_id']:
#         data[col] = lbl.fit_transform(data[col])
#     print('user 0,1 feature')
#     data['gender0'] = data['user_gender_id'].apply(lambda x: 1 if x == -1 else 2)
#     data['age0'] = data['user_age_level'].apply(lambda x: 1 if x == 1004 | x == 1005 | x == 1006 | x == 1007  else 2)
#     data['occupation0'] = data['user_occupation_id'].apply(lambda x: 1 if x == -1 | x == 2003  else 2)
#     data['star0'] = data['user_star_level'].apply(lambda x: 1 if x == -1 | x == 3000 | x == 3001  else 2)
#     print(
#         '--------------------------------------------------------------context--------------------------------------------------------------')
#     data['realtime'] = data['context_timestamp'].apply(timestamp_datetime)
#     data['realtime'] = pd.to_datetime(data['realtime'])
#     data['day'] = data['realtime'].dt.day
#     data['hour'] = data['realtime'].dt.hour
#     data['len_predict_category_property'] = data['predict_category_property'].map(lambda x: len(str(x).split(';')))
#     for i in range(5):
#         data['predict_category_property' + str(i)] = lbl.fit_transform(data['predict_category_property'].map(
#             lambda x: str(str(x).split(';')[i]) if len(str(x).split(';')) > i else ''))
#     print('context 0,1 feature')
#     data['context_page0'] = data['context_page_id'].apply(
#         lambda x: 1 if x == 4001 | x == 4002 | x == 4003 | x == 4004 | x == 4007  else 2)
#     print(
#         '--------------------------------------------------------------shop--------------------------------------------------------------')
#     for col in ['shop_id']:
#         data[col] = lbl.fit_transform(data[col])
#     data['shop_score_delivery0'] = data['shop_score_delivery'].apply(lambda x: 0 if x <= 0.98 and x >= 0.96  else 1)
    return data

In [21]:
new_data = base_process(data)
new_data.head()

--------------------------------------------------------------item--------------------------------------------------------------


,instance_id,item_id,item_category_list,item_property_list,item_brand_id,item_city_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,...,item_property_list0,item_property_list1,item_property_list2,item_property_list3,item_property_list4,item_property_list5,item_property_list6,item_property_list7,item_property_list8,item_property_list9
0,108641074714126964,3760,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,448,50,3,3,4,14,...,27,66,40,226,256,401,219,86,878,821
1,5754713551599725161,3760,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,448,50,3,3,4,14,...,27,66,40,226,256,401,219,86,878,821
2,842679481291040981,3760,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,448,50,3,3,4,14,...,27,66,40,226,256,401,219,86,878,821
3,937088850059189027,3760,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,448,50,3,3,4,14,...,27,66,40,226,256,401,219,86,878,821
4,7975697065017708072,3760,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,448,50,3,3,4,14,...,27,66,40,226,256,401,219,86,878,821


In [22]:
columns = new_data.columns
columns

Index(['instance_id', 'item_id', 'item_category_list', 'item_property_list',
       'item_brand_id', 'item_city_id', 'item_price_level', 'item_sales_level',
       'item_collected_level', 'item_pv_level', 'user_id', 'user_gender_id',
       'user_age_level', 'user_occupation_id', 'user_star_level', 'context_id',
       'context_timestamp', 'context_page_id', 'predict_category_property',
       'shop_id', 'shop_review_num_level', 'shop_review_positive_rate',
       'shop_star_level', 'shop_score_service', 'shop_score_delivery',
       'shop_score_description', 'is_trade', 'len_item_category',
       'len_item_property', 'item_category_list1', 'item_category_list2',
       'item_property_list0', 'item_property_list1', 'item_property_list2',
       'item_property_list3', 'item_property_list4', 'item_property_list5',
       'item_property_list6', 'item_property_list7', 'item_property_list8',
       'item_property_list9'],
      dtype='object')

In [23]:
data[['item_id', 'item_brand_id', 'item_city_id']]

,item_id,item_brand_id,item_city_id
0,3760,448,50
1,3760,448,50
2,3760,448,50
3,3760,448,50
4,3760,448,50
5,3760,448,50
6,333,2011,9
7,333,2011,9
8,5703,1257,9
9,333,2011,9
